<a href="https://colab.research.google.com/github/xHexlabx/SuperAI_SS4_Recap/blob/main/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/Liver_Ultrasound_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SuperAI Season 4 - Level 2 Hackathon - Liver_Ultrasound_Detection

## Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Explore Datasets

### Load Datasets

In [ ]:
!unzip /content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver-ultrasound-detection.zip

### Explore Train Images Directory

In [3]:
import os

In [4]:
n_images = len(os.listdir('/content/train/train/images'))
n_lesions = len(os.listdir('/content/train/train/annotations'))
n_non_lesions = n_images - n_lesions

print('num of datas =' , n_images , 'num of lesion =', n_lesions , 'num of non lesion' , n_non_lesions)

num of datas = 14448 num of lesion = 7222 num of non lesion 7226


### Explore Train Annotations Directory

In [5]:
annotations_path = '/content/train/train/annotations'

with open (f'{annotations_path}/{100022}.txt' , 'r') as file :

    file_content = file.read().split(' ')

    print(f'class = {file_content[0]}\ncenter_x = {file_content[1]}\ncenter_y = {file_content[2]}\nbox_width / image_width = {file_content[3]}\nbox_height / image_height = {file_content[4]}')

class = 2
center_x = 0.4601851851851852
center_y = 0.30185185185185187
box_width / image_width = 0.08703703703703704
box_height / image_height = 0.12222222222222222



## Setup Format Datasets For YOLOv10

### Define Path Of Directories

In [6]:
import os
import pandas as pd

In [7]:
train_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/train'
validation_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/validation'
test_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/test'

In [8]:
train_labels_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/labels/train'
validation_labels_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/labels/validation'

### Managing With Annotation Mapping File

In [9]:
annotation_mapping_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotation_mapping.csv'
annotation_mapping = pd.read_csv(annotation_mapping_path)

In [10]:
annotation_mapping

,Image File,Annotation File,Source,Type
0,24084.jpg,24084.txt,machine,machine_positive
1,97528.jpg,97528.txt,machine,machine_positive
2,54563.jpg,54563.txt,machine,machine_positive
3,115211.jpg,115211.txt,machine,machine_positive
4,71913.jpg,71913.txt,machine,machine_positive
...,...,...,...,...
19341,1160.jpg,NaN,mobile,mobile_negative
19342,153.jpg,NaN,mobile,mobile_negative
19343,130705.jpg,NaN,mobile,mobile_negative
19344,812.jpg,NaN,mobile,mobile_negative


In [11]:
annotation_mapping_trainable = annotation_mapping[annotation_mapping['Annotation File'].isnull() == False]

In [12]:
annotation_mapping_trainable

,Image File,Annotation File,Source,Type
0,24084.jpg,24084.txt,machine,machine_positive
1,97528.jpg,97528.txt,machine,machine_positive
2,54563.jpg,54563.txt,machine,machine_positive
3,115211.jpg,115211.txt,machine,machine_positive
4,71913.jpg,71913.txt,machine,machine_positive
...,...,...,...,...
18662,31309.jpg,31309.txt,mobile,mobile_positive
18663,103330.jpg,103330.txt,mobile,mobile_positive
18664,141103.jpg,141103.txt,mobile,mobile_positive
18665,9070.jpg,9070.txt,mobile,mobile_positive


In [13]:
print(f'num of all files = {len(annotation_mapping)} num of trainable files = {len(annotation_mapping_trainable)}')

num of all files = 19346 num of trainable files = 9667


### Create annotation_for_datasets.csv file

In [33]:
import os
import pandas as pd
from PIL import Image
import numpy as np
from tqdm import tqdm

In [34]:
annotation_for_train_datasets = {

    'file_name' : [] ,
    'width' : [] ,
    'height' : [] ,
    'class' : [] ,

}
#class 0 -> no lesion 1 -> lesion

In [35]:
# loop in train
train_dataset_path = '/content/train/train'

train_dataset_annotations = os.listdir(f'{train_dataset_path}/annotations')

for idx , file_name in tqdm(enumerate(os.listdir(f'{train_dataset_path}/images'))) :

    name = file_name.split('.')[0]

    annotation_for_train_datasets['file_name'].append(name)

    class_ = 1 if f'{name}.txt' in train_dataset_annotations else 0
    annotation_for_train_datasets['class'].append(class_)

    image = Image.open(f'{train_dataset_path}/images/{file_name}')

    shape = np.array(image).shape

    annotation_for_train_datasets['width'].append(shape[0])
    annotation_for_train_datasets['height'].append(shape[1])

14448it [02:22, 101.19it/s]


In [48]:
annotation_for_train_datasets_df = pd.DataFrame.from_dict(annotation_for_train_datasets)
annotation_for_train_datasets_df.set_index('file_name', inplace=True)

In [49]:
annotation_for_train_datasets_df

,width,height,class
file_name,,,
98356,782,1042,1
74796,810,1080,0
121413,810,1080,1
148426,540,720,1
122997,540,720,0
...,...,...,...
69995,540,720,1
116763,810,1080,0
106056,810,1080,1


In [50]:
annotation_for_train_datasets_df.to_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotation_for_train_datasets.csv')

In [51]:
annotation_for_train_datasets_df = pd.read_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotation_for_train_datasets.csv' , index_col = 'file_name')
annotation_for_train_datasets_df

,width,height,class
file_name,,,
98356,782,1042,1
74796,810,1080,0
121413,810,1080,1
148426,540,720,1
122997,540,720,0
...,...,...,...
69995,540,720,1
116763,810,1080,0
106056,810,1080,1


In [52]:
annotation_for_validation_datasets = {

    'file_name' : [] ,
    'width' : [] ,
    'height' : [] ,
    'class' : [] ,

}
#class 0 -> no lesion 1 -> lesion

In [53]:
# loop in validation
validation_dataset_path = '/content/val/val'

validation_dataset_annotations = os.listdir(f'{validation_dataset_path}/annotations')

for idx , file_name in tqdm(enumerate(os.listdir(f'{validation_dataset_path}/images'))) :

    name = file_name.split('.')[0]

    annotation_for_validation_datasets['file_name'].append(name)
    class_ = 1 if f'{name}.txt' in validation_dataset_annotations else 0

    annotation_for_validation_datasets['class'].append(class_)

    image = Image.open(f'{validation_dataset_path}/images/{file_name}')

    shape = np.array(image).shape

    annotation_for_validation_datasets['width'].append(shape[0])
    annotation_for_validation_datasets['height'].append(shape[1])

4898it [02:42, 30.19it/s]


In [54]:
annotation_for_validation_datasets_df = pd.DataFrame.from_dict(annotation_for_validation_datasets)
annotation_for_validation_datasets_df.set_index('file_name', inplace=True)

In [55]:
annotation_for_validation_datasets_df

,width,height,class
file_name,,,
27908,540,720,0
77411,540,720,0
48899,540,720,1
23485,810,1080,1
129834,614,816,0
...,...,...,...
86856,3024,4032,1
1684,3024,4032,0
10519,660,880,0


In [56]:
annotation_for_validation_datasets_df.to_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotation_for_validation_datasets.csv')

In [58]:
annotation_for_validation_datasets_df = pd.read_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotation_for_validation_datasets.csv' , index_col = 'file_name')
annotation_for_validation_datasets_df

,width,height,class
file_name,,,
27908,540,720,0
77411,540,720,0
48899,540,720,1
23485,810,1080,1
129834,614,816,0
...,...,...,...
86856,3024,4032,1
1684,3024,4032,0
10519,660,880,0


### Create Images / Labels Folder

In [66]:
import cv2
import os
from tqdm import tqdm

In [67]:
train_dataset_images_path = '/content/train/train/images'
validation_dataset_images_path = '/content/val/val/images'
test_dataset_images_path = '/content/test/test/images'

In [68]:
new_train_dataset_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/train'
new_validation_dataset_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/validation'
new_test_dataset_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/test'

In [70]:
for file_name in tqdm(os.listdir(train_dataset_images_path)) :

    image = cv2.imread(f'{train_dataset_images_path}/{file_name}')
    cv2.imwrite(f'{new_train_dataset_images_path}/{file_name}' , image)

100%|██████████| 14448/14448 [11:17<00:00, 21.32it/s]


In [71]:
for file_name in tqdm(os.listdir(validation_dataset_images_path)) :

    image = cv2.imread(f'{validation_dataset_images_path}/{file_name}')
    cv2.imwrite(f'{new_validation_dataset_images_path}/{file_name}' , image)

100%|██████████| 4898/4898 [12:36<00:00,  6.47it/s]


In [72]:
for file_name in tqdm(os.listdir(test_dataset_images_path)) :

    image = cv2.imread(f'{test_dataset_images_path}/{file_name}')
    cv2.imwrite(f'{new_test_dataset_images_path}/{file_name}' , image)

100%|██████████| 5153/5153 [33:36<00:00,  2.55it/s]


## Dinov2 For Classification

### Basic Use Of Dinov2

In [ ]:
from transformers import AutoImageProcessor, Dinov2ForImageClassification
import torch
from datasets import load_dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [65]:
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base-imagenet1k-1-layer" , num_labels=2)
model = Dinov2ForImageClassification.from_pretrained("facebook/dinov2-base-imagenet1k-1-layer").to(device)

inputs = image_processor(image, return_tensors="pt").to(device)

with torch.no_grad():

    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
print(logits.shape)
print(inputs)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for huggingface/cats-image contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/huggingface/cats-image
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


torch.Size([1, 1000])


### Define Custom Dataset To lesion / non-lesion Classification

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
import pandas as pd
import numpy as np

In [ ]:
class DatasetForClassification(Dataset):

    def __init__(self, annotation_path , image_path, transform = None , target_transform = None):

        self.annotation = pd.read_csv(annotation_path)
        self.image_path = image_path
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):

        return len(os.listdir(self.image_path))

    def __getitem__(self, idx):

        image = read_image(f'{self.image_path}/{self.annotation.iloc[idx]['file_name']}.jpg')
        label = torch.tensor(self.annotation.iloc[idx]['class'] , dtype = torch.float)

        if self.transform:

            image = self.transform(image)

        if self.target_transform

            label = self.target_transform(label)

        return image , label

In [ ]:
transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
target_transform = Lambda(lambda y: torch.zeros(2, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))

In [ ]:
train_annotation_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotation_for_train_datasets.csv'
train_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/train'

In [ ]:
train_dataset = DatasetForClassification(train_annotation_path , train_images_path , transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
for images , labels in train_dataloader :

    print(images , labels)

## YOLOv10 For Object Detection

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda --version

conda 23.11.0


In [ ]:
!git clone https://github.com/THU-MIG/yolov10.git
%cd yolov10

fatal: destination path 'yolov10' already exists and is not an empty directory.
/content/yolov10


In [ ]:
!conda create -n yolov10 python=3.10.2
!conda activate yolov10
!pip install -r requirements.txt
!pip install -e .

Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/yolov10

  added / updated specs:
    - python=3.10.2


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  bzip2              conda-forge/linux-64::bzip2-1.0.8-hd590300_5 
  ca-certificates    conda-forge/linux-64::ca-certificates-2024.2.2-hbcca054_0 
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.40-hf3520f5_1 
  libffi             conda-forge/linux-64::libffi-3.4.2-h7f98852_5 
  libgcc-ng          conda-forge/linux-64::libgcc-ng-13.2.0-h77fa898_7 
  libgomp            conda-forge/linux-64::libgomp-13.2.0-h77

In [ ]:
data_yaml = """
train: /content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/train
val: /content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/validation
test: /content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/images/test
nc: 7
names: ['FCC' , 'FSS' , 'HCC' ,'cyst' , 'hemangioma' , 'dysplastic' , 'CCA']
"""

In [ ]:
data_yaml_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets'

with open (f'{data_yaml_path}/data.yaml' , 'w') as file :

    file.write(data_yaml)

## facebook/detr-resnet-50 For Object Detection

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests
import numpy as np

In [ ]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

In [ ]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
image

In [ ]:
np.array(image).shape

In [ ]:
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

In [ ]:
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):

    box = [round(i, 2) for i in box.tolist()]

    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

In [ ]:
!pip install timm

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("object-detection", model="facebook/detr-resnet-50")

pipe(image)